# Bonus: hyperparameter optimization using WandB Sweeps + Optuna

### !!! Warning
To be able to run the code below you need to have a WandB account

## Login to WandB

In [1]:
import wandb
import inspect
from wandb import CommError
import yaml

In [2]:

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mohammadbakir (wandb). Use `wandb login --relogin` to force relogin


True

In [17]:
PROJECT='diabetes-prediction'
ENTITY="mohammadbakir"

## Create a Lauch Queue


In [8]:
config = {"label": "hyperparams-finetune-optuna"}
api = wandb.Api()
try:
    queue = api.create_run_queue(
            entity="mohammadbakir",
            name="diabetes-prediction-queue",
            type="local-process",
            access="project",
            config=config,
        )
except CommError as e:
    print(e.message)

## Create the training job

In [9]:
!wandb job create -e "mohammadbakir" -p $PROJECT -n "xgb-classifier-diabetes" code ./ -E "xgb_job.py"

wandb: Creating launch job of type: code...
wandb: Adding directory to artifact (./.)... Done. 0.0s
wandb:                                                                                
wandb: Created job: mohammadbakir/diabetes-prediction/xgb-classifier-diabetes:v0, with alias: latest
wandb: View all jobs in project 'diabetes-prediction' here: https://wandb.ai/mohammadbakir/diabetes-prediction/jobs
wandb: 


## Create a function to optimize


In [10]:
import optuna 

def objective(trial):
    # Define search spaces for hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 10, 300)
    max_depth = trial.suggest_int('max_depth', 1, 20)
    min_child_weight = trial.suggest_float('min_child_weight', 0, 1)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1, log=True)

    print(f"{n_estimators=} {max_depth=} {min_child_weight=} {learning_rate=}")

    # !! don't actually train, return -1
    return -1    

### Test the conditional configuration function.

In [11]:
import optuna 
# Create an Optuna study
study = optuna.create_study(direction="maximize")

# Start the optimization process
study.optimize(objective, n_trials=2)


[I 2023-12-04 14:00:33,136] A new study created in memory with name: no-name-8b14b019-bdfd-4eec-a940-2becea731976
[I 2023-12-04 14:00:33,139] Trial 0 finished with value: -1.0 and parameters: {'n_estimators': 172, 'max_depth': 7, 'min_child_weight': 0.3089946568673373, 'learning_rate': 0.019633771018302524}. Best is trial 0 with value: -1.0.
[I 2023-12-04 14:00:33,140] Trial 1 finished with value: -1.0 and parameters: {'n_estimators': 137, 'max_depth': 19, 'min_child_weight': 0.13986777270822404, 'learning_rate': 0.38670637970740784}. Best is trial 0 with value: -1.0.


n_estimators=172 max_depth=7 min_child_weight=0.3089946568673373 learning_rate=0.019633771018302524
n_estimators=137 max_depth=19 min_child_weight=0.13986777270822404 learning_rate=0.38670637970740784


## Save the configuration to W&B as an artifact.

Save the conditional search logic to W&B as an artifact.

In [12]:
ARTIFACT_FILENAME = "optuna_diabetes_prediction.py"
ARTIFACT_NAME = "optuna-config-diabetes-prediction"


In [14]:

"""write function to its own file"""
function_lines = inspect.getsource(objective)
with open(ARTIFACT_FILENAME, 'w') as f:
    f.write(function_lines)

"""create and log artifact to wandb"""
run = wandb.init(entity="mohammadbakir",project=PROJECT)
artifact = run.log_artifact(ARTIFACT_FILENAME, name=ARTIFACT_NAME, type='optuna')
run.finish()

wandb: Currently logged in as: mohammadbakir. Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


### The following way of creating the scheduler fails:

In [21]:
!wandb job create --project diabetes-prediction --name "optuna-scheduler" git https://github.com/wandb/launch-jobs --entry-point "jobs/sweep_schedulers/optuna_scheduler/optuna_scheduler.py"

wandb: Creating launch job of type: git...
wandb: ERROR Could not find requirements.txt file in git repo at https://github.com/wandb/requirements.txt or parent directories.
wandb: ERROR Job creation failed


So I copied the file from https://github.com/wandb/launch-jobs/blob/main/jobs/sweep_schedulers/optuna_scheduler/optuna_wandb.py into this project and created the job the following way

In [15]:
!wandb job create --entity="mohammadbakir" --project diabetes-prediction --name "optuna-scheduler" code ./ -E "optuna_scheduler.py"

wandb: Creating launch job of type: code...
wandb: Adding directory to artifact (./.)... Done. 0.0s
wandb:                                                                                
wandb: Created job: mohammadbakir/diabetes-prediction/optuna-scheduler:v0, with alias: latest
wandb: View all jobs in project 'diabetes-prediction' here: https://wandb.ai/mohammadbakir/diabetes-prediction/jobs
wandb: 


In [16]:
!wandb job create --entity "mohammadbakir" --project diabetes-prediction --name "optuna-scheduler" git https://github.com/MBakirWB/launch_testing \
     --entry-point "optuna-wandb-sweeps-hyperparameter-tuning/optuna_scheduler.py"

wandb: Creating launch job of type: git...
wandb: Using requirements.txt in /
wandb:                                                                                
wandb: Updated job: mohammadbakir/diabetes-prediction/optuna-scheduler:v1, with alias: latest
wandb: View all jobs in project 'diabetes-prediction' here: https://wandb.ai/mohammadbakir/diabetes-prediction/jobs
wandb: 


## Define a sweep configuration

In [20]:
config = {
    "metric": {"name": "validation_0-custom_recall_score", "goal": "maximize"},
    "run_cap": 4,
    "job": f"{ENTITY}/diabetes-prediction/xgb-classifier-diabetes:latest",
    "scheduler": {
        "job": f"{ENTITY}/diabetes-prediction/optuna-scheduler:latest",
        "num_workers": 2,
        "settings": {
            "optuna_source": f"{ENTITY}/{PROJECT}/{ARTIFACT_NAME}:latest",
            "optuna_source_filename": ARTIFACT_FILENAME,
        }
    },
}

# write config to file
config_filename = "sweep-config.yaml"
yaml.dump(config, open(config_filename, "w"))

## Launch the agent

Run from CLI

```shell
wandb launch-agent -q diabetes-prediction-queue
```

## Launch the sweep

In [21]:
! wandb launch-sweep sweep-config.yaml -e "mohammadbakir" -p $PROJECT -q diabetes-prediction-queue

wandb: WARNING Using a scheduler job for launch sweeps is *experimental* and may change without warning
wandb:   2 of 2 files downloaded.  


wandb: WARNING No program path found, not creating job artifact. See https://docs.wandb.ai/guides/launch/create-job


wandb:                                                                                
wandb:   2 of 2 files downloaded.  


wandb: WARNING No program path found, not creating job artifact. See https://docs.wandb.ai/guides/launch/create-job


wandb:                                                                                
wandb: launch: Launching run into mohammadbakir/diabetes-prediction
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. 'parameters' is a required property
wandb: Created sweep with ID: p1qne0to
wandb: View sweep at: https://wandb.ai/mohammadbakir/diabetes-prediction/sweeps/p1qne0to
wandb: Scheduler added to launch queue (diabetes-prediction-queue)
